# अध्याय 5 दस्तावेज़-आधारित प्रश्नोत्तर

 - [1. ओपनएआई एपीआई कुंजी सेट करें](#1. ओपनएआई-एपीआई-कुंजी सेट करें)
 - [1. आयात एम्बेडिंग मॉडल और वेक्टर भंडारण घटक] (#1. आयात एम्बेडिंग मॉडल और वेक्टर भंडारण घटक)
     - [1.1 वेक्टर स्टोरेज बनाएं](#1.1-वेक्टर स्टोरेज बनाएं)
     - [1.2 दस्तावेज़ों के साथ भाषा मॉडल का उपयोग करें](#1.2-दस्तावेज़ों के साथ भाषा मॉडल का उपयोग करें)
 - [2. हमारे दस्तावेज़ों से संबंधित प्रश्नों का उत्तर कैसे दें] (#2. - हमारे दस्तावेज़ों से संबंधित प्रश्नों का उत्तर कैसे दें)

एक सामान्य एप्लिकेशन परिदृश्य एक प्रश्न और उत्तर प्रणाली बनाने के लिए बड़े भाषा मॉडल का उपयोग करना है जो किसी दिए गए दस्तावेज़ और दस्तावेज़ संग्रह के बारे में प्रश्नों का उत्तर दे सकता है। किसी दिए गए दस्तावेज़ के लिए, जैसे कि पीडीएफ, वेब पेजों और कंपनी के होमपेजों से निकाला गया आंतरिक दस्तावेज़ संग्रह, हम इन दस्तावेज़ों की सामग्री के बारे में सवालों के जवाब देने के लिए बड़े भाषा मॉडल का उपयोग कर सकते हैं ताकि उपयोगकर्ताओं को उनकी ज़रूरत की जानकारी अधिक कुशलता से प्राप्त करने और उपयोग करने में मदद मिल सके . व्यावहारिक अनुप्रयोग परिदृश्यों के लिए यह विधि बहुत प्रभावी और लचीली है, क्योंकि यह न केवल बड़े भाषा मॉडल के मौजूदा प्रशिक्षण सेट डेटा जानकारी का उपयोग करती है, बल्कि यह बाहरी जानकारी का भी उपयोग कर सकती है।

इस प्रक्रिया में, हम लॉन्गचेन में अन्य घटकों को शामिल करेंगे, जैसे: एंबेडिंग मॉडल और वेक्टर स्टोर

## 1. OpenAI API कुंजी सेट करें

एपीआई कुंजी प्राप्त करने के लिए [ओपनएआई खाते] (https://platform.openai.com/account/api-keys) में लॉग इन करें, और फिर इसे एक पर्यावरण चर के रूप में सेट करें।

- यदि आप इसे वैश्विक पर्यावरण चर के रूप में सेट करना चाहते हैं, तो आप [ज़िहु लेख] (https://zhuanlan.zhihu.com/p/627665725) का संदर्भ ले सकते हैं।
- यदि आप इसे स्थानीय/प्रोजेक्ट पर्यावरण चर के रूप में सेट करना चाहते हैं, तो इस फ़ाइल निर्देशिका में एक `.env` फ़ाइल बनाएं, फ़ाइल खोलें और निम्नलिखित सामग्री दर्ज करें।

    <p style=”font-family:verdana; फ़ॉन्ट-आकार:12px;रंग:हरा”>
    OPENAI_API_KEY='आपका_api_key'
    </p>
  
  "your_api_key" को अपनी स्वयं की API कुंजी से बदलें

In [1]:
# आवश्यक पैकेज Python-dotenv और openai डाउनलोड करें
# यदि आपको इंस्टॉलेशन प्रक्रिया लॉग देखने की आवश्यकता है, तो आप -q हटा सकते हैं
!pip install -q python-dotenv
!pip install -q openai

In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# स्थानीय/परियोजना पर्यावरण चर पढ़ें।

# find_dotenv() .env फ़ाइल का पथ ढूंढता है और उसका पता लगाता है
#load_dotenv() .env फ़ाइल को पढ़ता है और उसमें मौजूद पर्यावरण चर को वर्तमान चल रहे वातावरण में लोड करता है
# यदि आप एक वैश्विक पर्यावरण चर सेट करते हैं, तो कोड की इस पंक्ति का कोई प्रभाव नहीं पड़ता है।
_ = load_dotenv(find_dotenv())

# पर्यावरण चर OPENAI_API_KEY प्राप्त करें
openai.api_key = os.environ['OPENAI_API_KEY']  

## 1. प्रत्यक्ष

In [25]:
!pip install --upgrade -q langchain
!pip install -q docarray

In [26]:
from langchain.chains import RetrievalQA #检索QA链，在文档上进行检索
from langchain.chat_models import ChatOpenAI #openai模型
from langchain.document_loaders import CSVLoader #文档加载器，采用csv格式存储
from langchain.vectorstores import DocArrayInMemorySearch #向量存储
from IPython.display import display, Markdown #在jupyter显示信息的工具

### 1.1 डेटा आयात करें

In [6]:
file = 'data/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [28]:
#पांडा का उपयोग करके डेटा आयात करें और देखें
import pandas as pd

data = pd.read_csv(file,usecols=[1, 2])
data.head()

,name,description
0,Women's Campside Oxfords,This ultracomfortable lace-to-toe Oxford boast...
1,"Recycled Waterhog Dog Mat, Chevron Weave",Protect your floors from spills and splashing ...
2,Infant and Toddler Girls' Coastal Chill Swimsu...,"She'll love the bright colors, ruffles and exc..."
3,"Refresh Swimwear, V-Neck Tankini Contrasts",Whether you're going for a swim or heading out...
4,EcoFlex 3L Storm Pants,Our new TEK O2 technology makes our four-seaso...


आउटडोर कपड़ों की एक सीएसवी फ़ाइल प्रदान की गई है जिसका उपयोग हम भाषा मॉडल के साथ संयोजन में करेंगे

### 1.2 मूल दस्तावेज़ लोडर वेक्टर भंडारण बनाता है

In [29]:
from langchain.indexes import VectorstoreIndexCreator #导入向量存储索引创建器

In [32]:
#निर्दिष्ट वेक्टर स्टोरेज क्लास बनाएं
# निर्माण के बाद, इसे लोडर से कॉल करें और दस्तावेज़ लोडर सूची के माध्यम से लोड करें
index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch).from_loaders([loader])

### 1.3 निर्मित वेक्टर स्टोरेज को क्वेरी करें

In [33]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [34]:
response = index.query(query)#使用索引查询创建一个响应，并传入这个查询

In [35]:
display(Markdown(response))#查看查询返回的内容



| Name | Description |
| --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+ rated, 100% polyester, wrinkle-resistant, front and back cape venting, two front bellows pockets |
| Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+ rated, 52% polyester and 48% nylon, machine washable and dryable, front and back cape venting, two front bellows pockets |
| Men's TropicVibe Shirt, Short-Sleeve | UPF 50+ rated, 71% Nylon, 29% Polyester, 100% Polyester knit mesh, wrinkle resistant, front and back cape venting, two front bellows pockets |
| Sun Shield Shirt by | UPF 50+ rated, 78% nylon, 22% Lycra Xtra Life fiber, wicks moisture, fits comfortably over swimsuit, abrasion resistant |

All four shirts provide UPF 50+ sun protection, blocking 98% of the sun's harmful rays. The Men's Tropical Plaid Short-Sleeve Shirt is made of 100% polyester and is wrinkle-resistant. The Men's Plaid Trop

धूप से सुरक्षा वाली सभी शर्टों के नाम और विवरण के साथ एक मार्कडाउन तालिका और भाषा मॉडल द्वारा प्रदान किया गया एक अच्छा सा सारांश मिला।

## 2. प्रतिनिधित्व मॉडल और वेक्टर भंडारण को मिलाएं

जब किसी दस्तावेज़ के साथ संयोजन में उपयोग किया जाता है, तो भाषा मॉडल एक समय में केवल कुछ हज़ार शब्दों की जानकारी का उपयोग कर सकता है। यदि हमारा दस्तावेज़ लंबा है, तो हम उसकी सभी सामग्री के बारे में सवालों के जवाब देने के लिए भाषा मॉडल कैसे प्राप्त कर सकते हैं? हम इसे कई वेक्टर अभ्यावेदन (एंबेडिंग) और वेक्टर स्टोरेज (वेक्टर स्टोर) के माध्यम से कार्यान्वित करते हैं।

पाठ प्रतिनिधित्व (एंबेडिंग) पाठ शब्दार्थ का एक वेक्टर प्रतिनिधित्व है। समान सामग्री वाले पाठों में समान प्रतिनिधित्व वेक्टर होते हैं। यह हमें वेक्टर स्पेस में टेक्स्ट की समानता की तुलना करने की अनुमति देता है।

वेक्टर डेटाबेस का उपयोग दस्तावेज़ों के टेक्स्ट ब्लॉक को संग्रहीत करने के लिए किया जाता है। किसी दिए गए दस्तावेज़ के लिए, हम पहले इसे छोटे पाठ खंडों में विभाजित करते हैं, फिर प्रत्येक खंड के लिए पाठ प्रतिनिधित्व प्राप्त करते हैं और इन प्रस्तुतीकरणों को एक वेक्टर डेटाबेस में संग्रहीत करते हैं। यह प्रक्रिया बिल्कुल एक इंडेक्स बनाने की प्रक्रिया है। दस्तावेज़ को पाठ के छोटे-छोटे टुकड़ों में तोड़ने का कारण यह है कि हम प्रसंस्करण के लिए पूरे दस्तावेज़ को भाषा मॉडल में पारित करने में सक्षम नहीं हो सकते हैं।

एक बार सूचकांक बन जाने के बाद, हम आने वाली क्वेरी (क्वेरी) के लिए सबसे अधिक प्रासंगिक पाठ अंशों को खोजने के लिए सूचकांक का उपयोग कर सकते हैं - पहले हमें क्वेरी के लिए एक वेक्टर प्रतिनिधित्व मिलता है, फिर हम वेक्टर डेटाबेस में सभी वैक्टरों के साथ इसकी तुलना करते हैं और चयन करते हैं सबसे समान एन टेक्स्ट ब्लॉक, और अंत में हम इन समान टेक्स्ट ब्लॉक से संकेत बनाते हैं और अंतिम उत्तर प्राप्त करने के लिए उन्हें भाषा मॉडल में इनपुट करते हैं।

### 2.1 डेटा आयात करें

In [7]:
#सीएसवी प्रारूप के माध्यम से लोड करने के लिए एक दस्तावेज़ लोडर बनाएं
loader = CSVLoader(file_path=file)
docs = loader.load()

In [8]:
#एकल दस्तावेज़ देखें, प्रत्येक दस्तावेज़ CSV में डेटा की एक पंक्ति से मेल खाता है
docs[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'data/OutdoorClothingCatalog_1000.csv', 'row': 0})

### 2.2 टेक्स्ट वेक्टर प्रतिनिधित्व मॉडल

In [9]:
#OpenAIEmbedding क्लास का उपयोग करें
from langchain.embeddings import OpenAIEmbeddings 
embeddings = OpenAIEmbeddings() #初始化

In [10]:
#क्योंकि दस्तावेज़ अपेक्षाकृत छोटा है, इसलिए यहां कोई खंडन करने की आवश्यकता नहीं है, वेक्टर प्रतिनिधित्व सीधे किया जा सकता है
#पाठ के लिए एक वेक्टर प्रतिनिधित्व बनाने के लिए प्रारंभिक OpenAIEmbedding उदाहरण पर क्वेरी विधि एम्बेड_क्वेरी का उपयोग करें
embed = embeddings.embed_query("Hi my name is Harrison")

#वेक्टर प्रतिनिधित्व की लंबाई जांचें
print(len(embed))

1536


In [11]:
#प्रत्येक तत्व एक अलग संख्यात्मक मान है, और संयोजन पाठ का एक वेक्टर प्रतिनिधित्व है।
print(embed[:5])

[-0.02194717898964882, 0.006735079921782017, -0.01816144771873951, -0.03916534036397934, -0.014086442068219185]


### 2.3 वेक्टर प्रतिनिधित्व के आधार पर वेक्टर स्टोरेज बनाएं

In [12]:
# अभी बनाए गए टेक्स्ट वेक्टर एम्बेडिंग को वेक्टर स्टोर में स्टोर करें
# लागू करने के लिए DocArrayInMemorySearch वर्ग की from_documents विधि का उपयोग करें
# यह विधि एक दस्तावेज़ सूची और एक वेक्टर प्रतिनिधित्व मॉडल को इनपुट के रूप में स्वीकार करती है
db = DocArrayInMemorySearch.from_documents(docs, embeddings)

In [28]:
import langchain
langchain.__version__

'0.0.234'

### 2.4 निर्मित वेक्टर स्टोरेज को क्वेरी करें

In [13]:
query = "Please suggest a shirt with sunblocking"

#आने वाली क्वेरी के समान टेक्स्ट ढूंढने और समान दस्तावेज़ों की सूची प्राप्त करने के लिए उपरोक्त वेक्टर स्टोरेज का उपयोग करें
docs = db.similarity_search(query)

In [14]:
#लौटे गए दस्तावेज़ों की संख्या प्रिंट करें
len(docs)

4

In [15]:
# लौटाए गए पहले दस्तावेज़ को प्रिंट करें
docs[0] 

Document(page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.', metadata={'source': 'data/OutdoorClothingCatalog_1000.csv', 'row': 255})

हम देख सकते हैं कि एक ने चार परिणाम लौटाए। पहला परिणाम आउटपुट सूरज से सुरक्षा के बारे में एक शर्ट है, जो हमारी क्वेरी की आवश्यकताओं को पूरा करता है: 'कृपया सनब्लॉकिंग वाली शर्ट का सुझाव दें)'

### 2.5 दस्तावेज़ों के बारे में प्रश्नों के उत्तर देने के लिए वेक्टर स्टोरेज का उपयोग करें

In [16]:
#वेक्टर स्टोरेज के आधार पर एक रिट्रीवर बनाएं
retriever = db.as_retriever() 

In [32]:
#एक बड़ा भाषा मॉडल आयात करें। यदि डिफ़ॉल्ट मॉडल gpt-3.5-टर्बो का उपयोग यहां किया जाता है, तो 504 सर्वर टाइमआउट होगा, इसलिए gpt-3.5-टर्बो-0301 का उपयोग किया जाता है।
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0301",temperature = 0.0) 

In [33]:
#प्राप्त समान दस्तावेज़ सामग्री को मर्ज करें
qdocs = "".join([docs[i].page_content for i in range(len(docs))])  

In [34]:
#मर्ज किए गए समान दस्तावेज़ सामग्री को प्रश्न के बाद `llm.call_as_llm` में इनपुट करें
#यहाँ प्रश्न यह है: मार्कडाउन तालिका में धूप से सुरक्षा फ़ंक्शन वाली सभी शर्टों की सूची बनाएं और उनका सारांश प्रस्तुत करें
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 

In [35]:
display(Markdown(response))

| Name | Description |
| --- | --- |
| Sun Shield Shirt | High-performance sun shirt with UPF 50+ sun protection, moisture-wicking, and abrasion-resistant fabric. Fits comfortably over swimsuits. |
| Men's Plaid Tropic Shirt | Ultracomfortable shirt with UPF 50+ sun protection, wrinkle-free fabric, and front/back cape venting. Made with 52% polyester and 48% nylon. |
| Men's TropicVibe Shirt | Men's sun-protection shirt with built-in UPF 50+ and wrinkle-resistant fabric. Features front/back cape venting and two front bellows pockets. |
| Men's Tropical Plaid Short-Sleeve Shirt | Lightest hot-weather shirt with UPF 50+ sun protection, relaxed traditional fit, and front/back cape venting. Made with 100% polyester. |

All of these shirts provide UPF 50+ sun protection, blocking 98% of the sun's harmful rays. They also have additional features such as moisture-wicking, wrinkle-resistant, and venting for cool breezes. The Sun Shield Shirt is abrasion-resistant and fits comfortably over swimsuits. The Men's Plaid Tropic Shirt is made with a blend of polyester and nylon and is machine washable/dryable. The Men's TropicVibe Shirt is also wrinkle-resistant and has two front bellows pockets. The Men's Tropical Plaid Short-Sleeve Shirt has a relaxed traditional fit and is made with 100% polyester.

यहां प्रतिक्रिया प्रिंट करने पर हम देख सकते हैं कि हमें बिल्कुल वैसा ही फॉर्म मिला जैसा हमने अनुरोध किया था

In [20]:
''' 
通过LangChain链封装起来
创建一个检索QA链，对检索到的文档进行问题回答，要创建这样的链，我们将传入几个不同的东西
1、语言模型，在最后进行文本生成
2、传入链类型，这里使用stuff，将所有文档塞入上下文并对语言模型进行一次调用
3、传入一个检索器
'''


qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [21]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."#创建一个查询并在此查询上运行链

In [22]:
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...


KeyboardInterrupt: 

In [58]:
display(Markdown(response))#使用 display 和 markdown 显示它



| Name | Description |
| --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+ rated, 100% polyester, wrinkle-resistant, front and back cape venting, two front bellows pockets |
| Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+ rated, 52% polyester and 48% nylon, machine washable and dryable, front and back cape venting, two front bellows pockets |
| Men's TropicVibe Shirt, Short-Sleeve | UPF 50+ rated, 71% Nylon, 29% Polyester, 100% Polyester knit mesh, machine wash and dry, front and back cape venting, two front bellows pockets |
| Sun Shield Shirt by | UPF 50+ rated, 78% nylon, 22% Lycra Xtra Life fiber, handwash, line dry, wicks moisture, fits comfortably over swimsuit, abrasion resistant |

All four shirts provide UPF 50+ sun protection, blocking 98% of the sun's harmful rays. The Men's Tropical Plaid Short-Sleeve Shirt is made of 100% polyester and is wrinkle-resistant

दोनों विधियाँ समान परिणाम लौटाती हैं

यदि मैं कई अलग-अलग प्रकार के ब्लॉकों पर एक ही प्रकार का प्रश्नोत्तर करना चाहता हूँ तो मुझे क्या करना चाहिए? पिछले प्रयोग में, केवल 4 दस्तावेज़ लौटाए गए थे, यदि एकाधिक दस्तावेज़ हैं, तो हम कई अलग-अलग तरीकों का उपयोग कर सकते हैं
* मानचित्र छोटा करना
प्रश्न के साथ सभी हिस्सों को भाषा मॉडल में पास करें, प्रतिक्रियाएँ प्राप्त करें, सभी व्यक्तिगत प्रतिक्रियाओं को अंतिम उत्तर में सारांशित करने के लिए किसी अन्य भाषा मॉडल कॉल का उपयोग करें, इसे किसी भी संख्या में दस्तावेज़ों पर चलाया जा सकता है। अधिक कॉल की आवश्यकता होने पर एक ही समस्या को समानांतर में संसाधित किया जा सकता है। यह सभी दस्तावेज़ों को स्वतंत्र मानता है
* परिष्कृत करें
कई दस्तावेज़ों पर लूप करने के लिए उपयोग किया जाता है, वास्तव में, पिछले दस्तावेज़ों के उत्तरों पर निर्माण करना, कारण संबंधी जानकारी के लिए आदर्श और समय के साथ क्रमिक रूप से उत्तरों का निर्माण करना, पिछले कॉल के परिणामों पर भरोसा करना। इसमें आमतौर पर अधिक समय लगता है और मूल रूप से मैप रिड्यूस जितनी कॉल की आवश्यकता होती है
* मानचित्र पुनः रैंक करें
प्रति दस्तावेज़ एक एकल भाषा मॉडल कॉल करें, उसे एक स्कोर लौटाने के लिए कहें, उच्चतम स्कोर चुनें, यह भाषा मॉडल पर निर्भर करता है कि स्कोर क्या होना चाहिए, आपको यह बताना होगा कि यदि यह दस्तावेज़ के लिए प्रासंगिक है तो यह होना चाहिए एक उच्च स्कोर, और वहाँ बारीकी से ट्यून किए गए निर्देश, उन्हें अपेक्षाकृत जल्दी बैचों में संसाधित किया जा सकता है, लेकिन अधिक महंगा है
*सामग्री
सभी चीज़ों को एक दस्तावेज़ में संयोजित करें